## Get Player Season Data

In [ ]:
import pandas as pd

In [ ]:
# Use nba_api (https://github.com/swar/nba_api?tab=readme-ov-file)
!pip install nba_api
!pip install numpy --upgrade
from nba_api.stats.endpoints import LeagueDashPlayerStats

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.9/284.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 49.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.


In [ ]:
# Get NBA player stats for the current season
playerStats = LeagueDashPlayerStats(per_mode_detailed='PerGame', season='2024-25', measure_type_detailed_defense='Advanced')

# Convert to pandas DataFrame
playerSeasonStats = playerStats.get_data_frames()[0]
# Drop irrelevant columns data
playerSeasonStats = playerSeasonStats.drop(columns=['PLAYER_ID', 'NICKNAME', 'TEAM_ID'])
playerSeasonStats = playerSeasonStats.reset_index(drop=True)

In [ ]:
# Convert non-numeric columns to numeric if possible
for col in playerSeasonStats.columns:
  try:
    playerSeasonStats[col] = pd.to_numeric(playerSeasonStats[col])
  except ValueError:
    pass

In [ ]:
playerSeasonStats.to_csv('playerSeasonStats.csv', index=False)

## Get Player Game by Game Data

In [ ]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog

In [ ]:
import time

# Retrieve all active NBA players
nba_players = players.get_active_players()

# Initialize a list to hold DataFrames
gamelog_dfs = []

# Iterate over each player to fetch their game logs
for player in nba_players:
    player_id = player['id']
    player_name = player['full_name']

    # Retrieve the player's game logs for the current season
    try:
        gamelog = playergamelog.PlayerGameLog(player_id=player_id, season='2024-25')
        gamelog_df = gamelog.get_data_frames()[0]

        # Add the player's name to the DataFrame
        gamelog_df['PLAYER_NAME'] = player_name

        # Append the DataFrame to the list if it's not empty or all-NA
        if not gamelog_df.empty and not gamelog_df.isna().all().all():
            gamelog_dfs.append(gamelog_df)

        # Pause to prevent rate limiting
        time.sleep(0.6)

    except Exception as e:
        print(f"Error retrieving data for {player_name}: {e}")

# Concatenate all DataFrames
if gamelog_dfs:
    all_players_gamelogs = pd.concat(gamelog_dfs, ignore_index=True)
else:
    all_players_gamelogs = pd.DataFrame()

In [ ]:
#filter
all_players_gamelogs['GAME_DATE'] = pd.to_datetime(all_players_gamelogs['GAME_DATE'])
all_players_gamelogs = all_players_gamelogs.drop(columns=['SEASON_ID', 'Player_ID', 'Game_ID', 'MATCHUP', 'VIDEO_AVAILABLE'])

<ipython-input-5-dc905865f840>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_players_gamelogs['GAME_DATE'] = pd.to_datetime(all_players_gamelogs['GAME_DATE'])


In [ ]:
all_players_gamelogs.to_csv('all_players_gamelogs.csv', index=False)
all_players_gamelogs.dtypes

,0
GAME_DATE,datetime64[ns]
WL,object
MIN,int64
FGM,int64
FGA,int64
FG_PCT,float64
FG3M,int64
FG3A,int64
FG3_PCT,float64
FTM,int64


## Get Team Season Data

In [ ]:
%%capture output
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install -q selenium

In [ ]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

service = webdriver.chrome.service.Service("/usr/bin/chromedriver")

driver = webdriver.Chrome(service=service, options=chrome_options)


In [ ]:
from bs4 import BeautifulSoup
import pandas as pd

driver.get('https://www.espn.com/nba/stats/team/_/season/2025/seasontype/2')
soup = BeautifulSoup(driver.page_source, 'html.parser')

table = soup.find('table', class_='Table Table--align-right Table--fixed Table--fixed-left', style="border-collapse:collapse;border-spacing:0")
df = pd.read_html(str(table))[0]
df = df['Team']
table1 = soup.find('table', class_='Table Table--align-right', style="border-collapse:collapse;border-spacing:0")
df1 = pd.read_html(str(table1))[0]
teamSeasonData = pd.merge(df, df1, left_index=True, right_index=True)

<ipython-input-12-b9e593b80cae>:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
<ipython-input-12-b9e593b80cae>:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df1 = pd.read_html(str(table1))[0]


In [ ]:
# Adjust data for consistency
team_abbreviations = {
    "Memphis Grizzlies": "MEM",
    "Cleveland Cavaliers": "CLE",
    "Denver Nuggets": "DEN",
    "Oklahoma City Thunder": "OKC",
    "Atlanta Hawks": "ATL",
    "New York Knicks": "NYK",
    "Indiana Pacers": "IND",
    "Sacramento Kings": "SAC",
    "Chicago Bulls": "CHI",
    "Boston Celtics": "BOS",
    "Detroit Pistons": "DET",
    "Dallas Mavericks": "DAL",
    "Milwaukee Bucks": "MIL",
    "Phoenix Suns": "PHX",
    "San Antonio Spurs": "SAS",
    "Houston Rockets": "HOU",
    "Minnesota Timberwolves": "MIN",
    "Golden State Warriors": "GSW",
    "Los Angeles Lakers": "LAL",
    "Utah Jazz": "UTA",
    "LA Clippers": "LAC",
    "Toronto Raptors": "TOR",
    "Portland Trail Blazers": "POR",
    "New Orleans Pelicans": "NOP",
    "Philadelphia 76ers": "PHI",
    "Miami Heat": "MIA",
    "Washington Wizards": "WAS",
    "Charlotte Hornets": "CHA",
    "Brooklyn Nets": "BKN",
    "Orlando Magic": "ORL"
}

# Replace team names with abbreviations
teamSeasonData["Team"] = teamSeasonData["Team"].map(team_abbreviations)
teamSeasonData

,Team,GP,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OR,DR,REB,AST,STL,BLK,TO,PF
0,CLE,78,122.1,44.6,90.4,49.3,15.8,41.3,38.2,17.2,22.0,78.3,11.0,34.0,45.1,28.2,8.2,4.3,12.4,18.2
1,MEM,78,121.7,44.7,93.3,47.9,13.8,37.6,36.7,18.4,23.4,78.6,12.9,34.5,47.4,28.5,8.8,5.7,15.1,20.9
2,DEN,79,120.7,45.5,89.8,50.6,12.1,32.1,37.8,17.7,23.0,76.8,11.2,34.5,45.7,31.0,8.0,4.9,13.7,17.7
3,OKC,78,120.0,44.4,92.5,48.1,14.5,38.7,37.4,16.7,20.3,82.0,10.6,34.1,44.7,26.9,10.3,5.8,11.1,19.8
4,ATL,78,118.0,43.3,92.0,47.1,13.4,37.6,35.7,18.1,23.2,77.7,11.9,32.5,44.4,29.6,9.8,5.2,14.7,19.1
5,CHI,78,117.8,43.2,92.0,46.9,15.5,42.1,36.9,15.9,19.8,80.6,9.9,35.7,45.6,29.0,7.5,4.7,13.9,17.8
6,IND,78,117.5,43.7,88.9,49.2,13.3,35.7,37.2,16.8,21.2,79.1,8.9,32.5,41.4,29.5,8.5,5.3,12.2,18.7
7,BOS,78,116.9,41.8,90.1,46.3,17.9,48.3,37.0,15.5,19.3,80.1,11.4,33.9,45.3,26.2,7.1,5.5,11.3,16.0
8,NYK,78,116.1,43.5,89.1,48.8,12.5,33.8,37.0,16.7,20.8,80.2,10.8,32.0,42.8,27.5,8.1,3.9,12.2,17.0
9,SAC,78,115.9,43.0,90.1,47.7,12.6,35.4,35.6,17.2,21.5,80.3,10.9,33.3,44.2,26.5,7.7,4.4,12.8,19.0


In [ ]:
teamSeasonData.to_csv('teamSeasonData.csv', index=False)

## Get Team Game By Game Data

Team game by game

In [ ]:
import requests
import pandas as pd
import time
import numpy as np
from datetime import datetime
from tqdm import tqdm

# Mapping dictionary: full team names to NBA verified acronyms
TEAM_ACRONYMS = {
    'Atlanta Hawks': 'ATL',
    'Boston Celtics': 'BOS',
    'Brooklyn Nets': 'BKN',
    'Charlotte Hornets': 'CHA',
    'Chicago Bulls': 'CHI',
    'Cleveland Cavaliers': 'CLE',
    'Dallas Mavericks': 'DAL',
    'Denver Nuggets': 'DEN',
    'Detroit Pistons': 'DET',
    'Golden State Warriors': 'GSW',
    'Houston Rockets': 'HOU',
    'Indiana Pacers': 'IND',
    'Los Angeles Clippers': 'LAC',
    'Los Angeles Lakers': 'LAL',
    'Memphis Grizzlies': 'MEM',
    'Miami Heat': 'MIA',
    'Milwaukee Bucks': 'MIL',
    'Minnesota Timberwolves': 'MIN',
    'New Orleans Pelicans': 'NOP',
    'New York Knicks': 'NYK',
    'Oklahoma City Thunder': 'OKC',
    'Orlando Magic': 'ORL',
    'Philadelphia 76ers': 'PHI',
    'Phoenix Suns': 'PHX',
    'Portland Trail Blazers': 'POR',
    'Sacramento Kings': 'SAC',
    'San Antonio Spurs': 'SAS',
    'Toronto Raptors': 'TOR',
    'Utah Jazz': 'UTA',
    'Washington Wizards': 'WAS'
}

# Months to scrape for each season
MONTHS = ['october', 'november', 'december', 'january', 'february', 'march', 'april', 'may', 'june']

def fetch_games_from_basketball_reference(seasons):
    """
    Fetch games from Basketball Reference for NBA teams for all months.
    Only include games from October 2022 onward.

    Parameters:
    seasons (list): List of season years to fetch (e.g. [2023, 2024, 2025] for 2022-23, 2023-24, 2024-25 seasons)
    """
    print(f"Fetching NBA games from Basketball Reference for the {', '.join([f'{year-1}-{str(year)[2:]}' for year in seasons])} seasons...")

    all_games = []
    # Define the start date (October 1, 2022)
    start_date = datetime(2024, 10, 1)

    for season in seasons:
        season_games_count = 0
        print(f"Processing {season-1}-{str(season)[2:]} season...")

        # Process each month for the season
        for month in MONTHS:
            try:
                # URL format: includes month in the URL
                url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games-{month}.html"
                print(f"Processing {month.capitalize()} for {season-1}-{str(season)[2:]} season from {url}")

                # Read the tables from the page
                try:
                    tables = pd.read_html(url)
                    if not tables:
                        print(f"No tables found for {month.capitalize()} {season}.")
                        continue

                    # The first table should be the schedule
                    schedule_df = tables[0]

                    # Remove rows where the Date is 'Playoffs' or other non-date values
                    schedule_df = schedule_df[schedule_df['Date'].str.contains(',', na=False)]

                    month_games_count = 0

                    # Process each game row
                    for _, row in schedule_df.iterrows():
                        date_str = row['Date']
                        visitor = row['Visitor/Neutral']
                        home = row['Home/Neutral']

                        # Replace full team names with acronyms
                        visitor_acronym = TEAM_ACRONYMS.get(visitor, visitor)
                        home_acronym = TEAM_ACRONYMS.get(home, home)

                        # Initialize default values for unplayed games
                        score = None
                        result = np.nan  # NaN for games not yet played

                        # Check if the game has been played by looking for scores
                        visitor_pts = row.get('PTS', None)
                        home_pts = row.get('PTS.1', None)

                        if pd.notna(visitor_pts) and pd.notna(home_pts):
                            try:
                                visitor_pts = int(visitor_pts)
                                home_pts = int(home_pts)
                                score = f"{visitor_pts}-{home_pts}"

                                # Determine result from home team's perspective
                                if home_pts > visitor_pts:
                                    result = "W"  # Home team won
                                else:
                                    result = "L"  # Home team lost
                            except Exception:
                                continue

                        # Parse the game date
                        try:
                            game_date = datetime.strptime(date_str, "%a, %b %d, %Y")
                        except Exception:
                            continue

                        # Include all games from the start date onward
                        if game_date >= start_date:
                            all_games.append({
                                'datetime': date_str,
                                'hometeam': home_acronym,
                                'awayteam': visitor_acronym,
                                'score': score,
                                'result': result
                            })
                            month_games_count += 1

                    print(f"Collected {month_games_count} games for {month.capitalize()} {season}")
                    season_games_count += month_games_count

                except Exception as e:
                    print(f"No data found for {month.capitalize()} {season} or error: {e}")

                time.sleep(1)  # Delay to avoid overwhelming the server

            except Exception as e:
                print(f"Error with {month.capitalize()} {season}: {e}")

        print(f"Total games collected for {season-1}-{str(season)[2:]} season: {season_games_count}")

    # Return DataFrame with only the desired columns
    df = pd.DataFrame(all_games)
    if not df.empty:
        df = df[['hometeam', 'awayteam', 'datetime', 'score', 'result']]

    return df

def analyze_team_coverage(df):
    """
    Analyze the dataset to check games per team (by acronym).
    """
    if df.empty:
        print("No data to analyze.")
        return

    # List of all NBA team acronyms from the mapping
    all_nba_teams = list(TEAM_ACRONYMS.values())

    print("\nGames per team:")
    for team in sorted(all_nba_teams):
        home_count = len(df[df['hometeam'] == team])
        away_count = len(df[df['awayteam'] == team])
        total_count = home_count + away_count
        print(f"{team}: {total_count} games")

    # Count games by season based on the datetime field
    if 'datetime' in df.columns:
        df['season'] = df['datetime'].apply(lambda x:
            f"{int(x.split(', ')[1].split(' ')[1]) - 1}-{x.split(', ')[1].split(' ')[1][2::]}"
            if len(x.split(', ')) > 1 and x.split(', ')[1].split(' ')[0] in ['Oct', 'Nov', 'Dec']
            else f"{int(x.split(', ')[1].split(' ')[1])}-{int(x.split(', ')[1].split(' ')[1]) + 1}"[2::]
        )

        print("\nGames per season:")
        season_counts = df['season'].value_counts().sort_index()
        for season, count in season_counts.items():
            print(f"{season}: {count} games")

def save_to_csv(df, filename="nba_games_schedule_2022_to_future.csv"):
    """
    Saves the DataFrame to a CSV file.

    Parameters:
    df (pandas.DataFrame): DataFrame to save
    filename (str): Output filename
    """
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

def main():
    # Seasons to fetch (e.g., 2022-23, 2023-24, 2024-25)
    seasons_to_fetch = [2025]

    # Fetch games from Basketball Reference
    games_df = fetch_games_from_basketball_reference(seasons_to_fetch)

    if games_df.empty:
        print("Failed to fetch NBA game data.")
        return

    # Display a sample of the data
    print("\nSample of the collected data:")
    print(games_df.head())

    # Display total number of games collected
    print(f"\nTotal number of games collected: {len(games_df)}")

    # Save to CSV
    save_to_csv(games_df)

    # Analyze team coverage
    analyze_team_coverage(games_df)

    return games_df

if __name__ == "__main__":
    df = main()

df.drop(columns=['season'], inplace=True)

Fetching NBA games from Basketball Reference for the 2024-25 seasons...
Processing 2024-25 season...
Processing October for 2024-25 season from https://www.basketball-reference.com/leagues/NBA_2025_games-october.html
Collected 71 games for October 2025
Processing November for 2024-25 season from https://www.basketball-reference.com/leagues/NBA_2025_games-november.html
Collected 222 games for November 2025
Processing December for 2024-25 season from https://www.basketball-reference.com/leagues/NBA_2025_games-december.html
Collected 192 games for December 2025
Processing January for 2024-25 season from https://www.basketball-reference.com/leagues/NBA_2025_games-january.html
Collected 227 games for January 2025
Processing February for 2024-25 season from https://www.basketball-reference.com/leagues/NBA_2025_games-february.html
Collected 176 games for February 2025
Processing March for 2024-25 season from https://www.basketball-reference.com/leagues/NBA_2025_games-march.html
Collected 238 

In [ ]:
teamSchedule = df.copy()
teamSchedule['datetime'] = pd.to_datetime(teamSchedule['datetime'])
teamSchedule.to_csv('teamSchedule.csv', index=False)
teamSchedule.dtypes

,0
hometeam,object
awayteam,object
datetime,datetime64[ns]
score,object
result,object


## Put Data in SQL Database

In [ ]:
!pip install -q mysqlclient
!pip install -q -U sql_magic
from sqlalchemy import create_engine
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.0 MB/s eta 0:00:00


In [ ]:
# Connect to the MySQL, but without selecting a database
conn_string = 'mysql://{user}:{password}@{host}:{port}/{db}?charset={encoding}'.format(
    user='Team_A',
    password='NkC121jpeTE=',
    host = 'jsedocc7.scrc.nyu.edu',
    port     = 3306,
    encoding = 'utf8',
    db = 'Team_A'
)
engine = create_engine(conn_string)

In [ ]:
# For Easy Access
playerSeasonStats = pd.read_csv('playerSeasonStats.csv')
all_players_gamelogs = pd.read_csv('all_players_gamelogs.csv')
all_players_gamelogs['GAME_DATE'] = pd.to_datetime(all_players_gamelogs['GAME_DATE'])
teamSeasonData = pd.read_csv('teamSeasonData.csv')
teamSeasonData.columns = teamSeasonData.columns.str.replace('%', 'Percent')
teamSeasonData = teamSeasonData.rename(columns={
    'OR': 'OffensiveRebounds',
    'TO': 'Turnovers'
})
teamSchedule = pd.read_csv('teamSchedule.csv')
teamSchedule['datetime'] = pd.to_datetime(teamSchedule['datetime'])

In [ ]:
from sqlalchemy import text

# Function to generate create table commands
def generate_create_table_sql(df, table_name):
    sql = ""
    sql += f"CREATE TABLE {table_name} (\n"

    for column in df.columns:
        dtype = str(df[column].dtype)
        if dtype == 'object':
            sql += f"  {column} VARCHAR(255),\n"
        elif dtype == 'int64':
            sql += f"  {column} INT,\n"
        elif dtype == 'float64':
            sql += f"  {column} DOUBLE,\n"
        elif 'datetime' in dtype:
            sql += f"  {column} DATETIME,\n"
        else:
            sql += f"  {column} VARCHAR(255),\n"

    # Add primary key for playerSeasonStats if needed.
    if table_name == "playerSeasonStats":
        sql += f"  PRIMARY KEY (PLAYER_NAME, TEAM_ABBREVIATION), \n"
        sql += f"  FOREIGN KEY (TEAM_ABBREVIATION) REFERENCES teamSeasonData(Team) ON DELETE CASCADE\n"
    elif table_name == 'teamSeasonData':
        sql += f"  PRIMARY KEY (Team) \n"
    elif table_name == 'all_players_gamelogs':
        sql += f"  PRIMARY KEY (PLAYER_NAME, GAME_DATE), \n"
        sql += f"  FOREIGN KEY (PLAYER_NAME) REFERENCES playerSeasonStats(PLAYER_NAME) ON DELETE CASCADE \n"
    elif table_name == 'teamSchedule':
        sql += f"  PRIMARY KEY (hometeam, awayteam, datetime), \n"
        sql += f"  FOREIGN KEY (hometeam) REFERENCES teamSeasonData(Team) ON DELETE CASCADE, \n"
        sql += f"  FOREIGN KEY (awayteam) REFERENCES teamSeasonData(Team) ON DELETE CASCADE\n"

    sql += ");\n"

    return sql

# Generate SQL create table commands for each dataframe
playerSeasonStats_sql = generate_create_table_sql(playerSeasonStats, 'playerSeasonStats')
all_players_gamelogs_sql = generate_create_table_sql(all_players_gamelogs, 'all_players_gamelogs')
teamSeasonData_sql = generate_create_table_sql(teamSeasonData, 'teamSeasonData')
teamSchedule_sql = generate_create_table_sql(teamSchedule, 'teamSchedule')

with engine.connect() as connection:
    connection.execute(text("DROP TABLE IF EXISTS all_players_gamelogs"))
    connection.execute(text("DROP TABLE IF EXISTS playerSeasonStats"))
    connection.execute(text("DROP TABLE IF EXISTS teamSchedule"))
    connection.execute(text("DROP TABLE IF EXISTS teamSeasonData"))
    connection.execute(text(teamSeasonData_sql))
    connection.execute(text(teamSchedule_sql))
    connection.execute(text(playerSeasonStats_sql))
    connection.execute(text(all_players_gamelogs_sql))

# Load dataframes to sql database
teamSeasonData.to_sql('teamSeasonData', con=engine, if_exists='append', index=False)
teamSchedule.to_sql('teamSchedule', con=engine, if_exists='append', index=False)
playerSeasonStats.to_sql('playerSeasonStats', con=engine, if_exists='append', index=False)
all_players_gamelogs.to_sql('all_players_gamelogs', con=engine, if_exists='append', index=False)

21978

In [ ]:
df = pd.read_sql("SELECT * FROM playerSeasonStats", con=engine)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555 entries, 0 to 554
Data columns (total 75 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   PLAYER_NAME              555 non-null    object 
 1   TEAM_ABBREVIATION        555 non-null    object 
 2   AGE                      555 non-null    float64
 3   GP                       555 non-null    int64  
 4   W                        555 non-null    int64  
 5   L                        555 non-null    int64  
 6   W_PCT                    555 non-null    float64
 7   MIN                      555 non-null    float64
 8   E_OFF_RATING             555 non-null    float64
 9   OFF_RATING               555 non-null    float64
 10  sp_work_OFF_RATING       555 non-null    float64
 11  E_DEF_RATING             555 non-null    float64
 12  DEF_RATING               555 non-null    float64
 13  sp_work_DEF_RATING       555 non-null    float64
 14  E_NET_RATING             5

In [ ]:
# Function to get primary keys for each table
def get_primary_keys(engine):
    query = """
    SELECT
        TABLE_NAME, COLUMN_NAME
    FROM
        INFORMATION_SCHEMA.COLUMNS
    WHERE
        COLUMN_KEY = 'PRI' AND TABLE_SCHEMA = DATABASE();
    """
    return pd.read_sql(query, engine)

# Function to get foreign keys for each table
def get_foreign_keys(engine):
    query = """
    SELECT
        TABLE_NAME, COLUMN_NAME, REFERENCED_TABLE_NAME, REFERENCED_COLUMN_NAME
    FROM
        INFORMATION_SCHEMA.KEY_COLUMN_USAGE
    WHERE
        TABLE_SCHEMA = DATABASE() AND REFERENCED_TABLE_NAME IS NOT NULL;
    """
    return pd.read_sql(query, engine)

# Get primary keys
primary_keys = get_primary_keys(engine)
print("\nPrimary Keys in the database:")
print(primary_keys)

# Get foreign keys
foreign_keys = get_foreign_keys(engine)
print("\nForeign Keys in the database:")
print(foreign_keys)



Primary Keys in the database:
             TABLE_NAME        COLUMN_NAME
0  all_players_gamelogs          GAME_DATE
1  all_players_gamelogs        PLAYER_NAME
2     playerSeasonStats        PLAYER_NAME
3     playerSeasonStats  TEAM_ABBREVIATION
4          teamSchedule           hometeam
5          teamSchedule           awayteam
6          teamSchedule           datetime
7        teamSeasonData               Team

Foreign Keys in the database:
             TABLE_NAME        COLUMN_NAME REFERENCED_TABLE_NAME  \
0  all_players_gamelogs        PLAYER_NAME     playerSeasonStats   
1     playerSeasonStats  TEAM_ABBREVIATION        teamSeasonData   
2          teamSchedule           hometeam        teamSeasonData   
3          teamSchedule           awayteam        teamSeasonData   

  REFERENCED_COLUMN_NAME  
0            PLAYER_NAME  
1                   Team  
2                   Team  
3                   Team  
